In [5]:
import pandas as pd

item_features = pd.read_csv("item_features.csv")
user_features = pd.read_csv("user_features.csv")
user_genre_freq = pd.read_csv("user_genre_freq.csv")
user_author_seen = pd.read_csv("user_author_seen.csv")

candidates_df = pd.read_csv("E:\Hakaton\participants\participants\submit\candidates.csv")

In [ ]:
import pandas as pd
from catboost import CatBoostRanker, Pool

# ==== 0) load model ====
model = CatBoostRanker()
model.load_model("model.cbm")

# ==== 1) build test_rank (таблица user_id, edition_id + ВСЕ фичи) ====
# ВАРИАНТ A: если у тебя УЖЕ есть готовая таблица со всеми фичами на candidate-пары:
# test_rank = your_ready_df.copy()

# ВАРИАНТ B: если у тебя есть candidates_df + витрины фич (item_features, user_features, user_genre_freq, user_author_seen):
test_rank = candidates_df.copy()

if "item_features" in globals():
    test_rank = test_rank.merge(item_features, on="edition_id", how="left")
if "user_features" in globals():
    test_rank = test_rank.merge(user_features, on="user_id", how="left")
if "user_genre_freq" in globals() and "genre_id" in test_rank.columns:
    test_rank = test_rank.merge(user_genre_freq, on=["user_id","genre_id"], how="left")
if "user_author_seen" in globals() and "author_id" in test_rank.columns:
    test_rank = test_rank.merge(user_author_seen, on=["user_id","author_id"], how="left")

# fill NaN (cold items / пропуски)
test_rank = test_rank.fillna(0)

# ==== 2) взять список фич ровно как в модели ====
feature_cols = model.feature_names_
X_test = test_rank[feature_cols]

# ==== 3) Pool + predict (ranker требует group_id + сортировку по user_id) ====
test_rank = test_rank.sort_values(["user_id", "edition_id"]).reset_index(drop=True)
X_test = test_rank[feature_cols]

test_pool = Pool(
    data=X_test,
    group_id=test_rank["user_id"],
)

test_rank["score"] = model.predict(test_pool)

# ==== 4) ранжирование внутри user и top-20 ====
test_rank = test_rank.sort_values(["user_id", "score"], ascending=[True, False])
test_rank["rank"] = test_rank.groupby("user_id").cumcount() + 1

result = test_rank.loc[test_rank["rank"] <= 20, ["user_id", "edition_id", "rank"]].copy()

# sanity: ровно 20 на каждого user (если у всех было >=20 кандидатов)
# assert result.groupby("user_id").size().eq(20).all()


,user_id,edition_id,rank
28,560,1003618326,1
9,560,1001514720,2
54,560,1006232332,3
53,560,1006232329,4
172,560,1010780717,5
...,...,...,...
1013333,11951620,1009914142,16
1013237,11951620,1004151598,17
1013357,11951620,1010755649,18
1013348,11951620,1010160292,19


In [12]:
result.reset_index()
result.to_csv("submit_pls_47.csv", index=False)